In [1]:
import tensorflow as tf
from tensorflow.python.keras.datasets import mnist
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Dropout, Flatten
from tensorflow.python.keras.layers import Conv2D, MaxPooling2D
from tensorflow.python.keras.estimator import model_to_estimator
import numpy as np
import os
print(tf.__version__)

/usr/local/envs/py3env/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


1.8.0


In [2]:
# input image dimensions
img_rows, img_cols = 28, 28
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

input_shape = (img_rows, img_cols, 1)
# convert class vectors to binary class matrices
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

In [3]:
model = Sequential()
model.add(Conv2D(20, (5, 5), activation='relu', padding='valid', input_shape=input_shape, name='x'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2))
model.add(Conv2D(40, (5, 5), activation='relu', padding='valid'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax', name='probabilities'))

model.compile(loss=tf.keras.losses.categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adam(0.002),
              metrics=['accuracy'])

In [4]:
model_dir = os.path.join(os.getcwd(), "models/mnist")
os.makedirs(model_dir, exist_ok=True)

In [5]:
estimator = model_to_estimator(keras_model=model, model_dir=model_dir)

INFO:tensorflow:Using the Keras model provided.
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/content/datalab/notebooks/CloudML/models/mnist', '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_is_chief': True, '_keep_checkpoint_every_n_hours': 10000, '_global_id_in_cluster': 0, '_save_summary_steps': 100, '_task_type': 'worker', '_session_config': None, '_keep_checkpoint_max': 5, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fb02150cc50>, '_save_checkpoints_steps': None, '_tf_random_seed': None, '_master': '', '_task_id': 0, '_service': None, '_evaluation_master': '', '_train_distribute': None, '_num_worker_replicas': 1, '_num_ps_replicas': 0}


In [6]:
train_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x_input": x_train},
      y=y_train,
      batch_size=128,
      num_epochs=None,
      shuffle=True)
train_spec = tf.estimator.TrainSpec(train_input_fn, max_steps=100) #for local

In [7]:
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x_input": x_test},
      y=y_test,
      num_epochs=1,
      shuffle=False)
eval_spec = tf.estimator.EvalSpec(eval_input_fn, steps=1)

In [8]:
tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after 600 secs (eval_spec.throttle_secs) or training is finished.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /content/datalab/notebooks/CloudML/models/mnist/keras_model.ckpt
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /content/datalab/notebooks/CloudML/models/mnist/model.ckpt.
INFO:tensorflow:step = 1, loss = 2.336228
INFO:tensorflow:Saving checkpoints for 85 into /content/datalab/notebooks/CloudML/models/mnist/model.ckpt.
INFO:tensorflow:Loss for final step: 0.33643314.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-10-15-05:05:18
INFO:te

In [9]:
num_images = 9
some_images = x_test[0:num_images]
some_images_pred = y_test[0:num_images]
predict_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x_input": some_images},
    num_epochs=1,
    shuffle=False)

In [10]:
predictions = estimator.predict(input_fn=predict_input_fn)
cls = [p['probabilities'] for p in predictions]

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /content/datalab/notebooks/CloudML/models/mnist/model.ckpt-100
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [12]:
for i in range(0,9):
    print('Actual:',np.argmax(some_images_pred[i]),'Pred:',np.argmax(cls[i]))

Actual: 7 Pred: 7
Actual: 2 Pred: 2
Actual: 1 Pred: 1
Actual: 0 Pred: 0
Actual: 4 Pred: 4
Actual: 1 Pred: 1
Actual: 4 Pred: 4
Actual: 9 Pred: 9
Actual: 5 Pred: 5
